# Introduction

Today we are going to test my custom implenetation of the ResNet18 from the famous paper: [Deep Residual Learning for Image Recognition](https://arxiv.org/abs/1512.03385). The Oxford IIIT pets [dataset](https://www.robots.ox.ac.uk/~vgg/data/pets/) was chosen to make a Cat vs. Dog classifier. 

# Imports

In [1]:
from fastai.vision.all import *
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from torchvision import transforms
import torch
from typing import List


from dataset_classes.pets_dataset import PetsDataset
from resnet.resnets import ResNet18


In [2]:
#!pip install -Uqqq torcheval

In [3]:
from torcheval.metrics import BinaryAccuracy

# Data preparation

In [4]:
data_path = untar_data(URLs.PETS)/'images'

In [5]:
fnames = get_image_files(data_path)
# getting the labels to stratify by later when splitting
labels = fnames.map(lambda x: 1 if x.name[0].isupper() else 0)

In [6]:
tr_transform = transforms.Compose([
    transforms.RandomResizedCrop((224,224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor()
])
vl_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor()
])


In [7]:
tr_files, vl_files = train_test_split(fnames, train_size=0.8, stratify=labels)

In [8]:
len(tr_files), len(vl_files)

(5912, 1478)

In [9]:
tr_ds = PetsDataset(tr_files, tr_transform)
vl_ds = PetsDataset(vl_files, vl_transform)

In [10]:
tr_dl = DataLoader(tr_ds, shuffle=True, batch_size=128)
vl_dl = DataLoader(vl_ds,  batch_size = 128)

# Modelling

In [11]:
model = ResNet18(in_channels=3, num_classes=2).to('cuda')

In [12]:
optim = torch.optim.AdamW(model.parameters(), 0.001)
loss_func = torch.nn.functional.binary_cross_entropy
metric = BinaryAccuracy()
ohe = torch.nn.functional.one_hot

In [13]:
def train_loop(epochs):
    best_value = float('-inf')
    for e in (range(epochs)):
        for xb, yb in progress_bar(tr_dl, comment=f'Train epoch {e+1}',leave=False):
            xb = xb[0].to('cuda')
            yb = ohe(yb[0].to('cuda'),2).float()
            preds = model(xb).softmax(dim=1)
            loss = loss_func(preds, yb)
            loss.backward()
            optim.step()
            optim.zero_grad()
        with torch.no_grad():
            vl_losses = []
            for xb, yb in progress_bar(vl_dl, comment=f'Validation epcoh #{e+1}', leave=False):
                xb = xb[0].to('cuda')
                yb_ohe = ohe(yb[0].to('cuda'),2).float()
                preds = model(xb).softmax(dim=1)
                loss = loss_func(preds, yb_ohe)
                vl_losses.append(loss.item())
                metric.update(preds.argmax(dim=1).to('cpu'), yb[0].to('cpu'))
        
        cur_value = metric.compute()
        print(f'Epoch #{e+1}: loss: {np.mean(vl_losses):.3f}, accuracy: {cur_value:.3f}')
        if cur_value > best_value:
            best_value = cur_value
            torch.save(model, 'resnet18-pets.pth')
            print(f'Found a better model at epoch {e+1}')
        metric.reset()

In [86]:
train_loop(30)

Epoch #1: loss: 0.586, accuracy: 0.677
Found a better model at epoch 1


Epoch #2: loss: 0.598, accuracy: 0.682
Found a better model at epoch 2


Epoch #3: loss: 0.578, accuracy: 0.684
Found a better model at epoch 3


Epoch #4: loss: 0.595, accuracy: 0.694
Found a better model at epoch 4


Epoch #5: loss: 0.544, accuracy: 0.704
Found a better model at epoch 5


...

Epoch #26: loss: 0.321, accuracy: 0.861


Epoch #27: loss: 0.332, accuracy: 0.851


Epoch #28: loss: 0.285, accuracy: 0.887
Found a better model at epoch 28


Epoch #29: loss: 0.326, accuracy: 0.867


Epoch #30: loss: 0.269, accuracy: 0.890
Found a better model at epoch 30


The model seems capable of learning more so we will train it further.

In [87]:
train_loop(30)

Epoch #1: loss: 0.311, accuracy: 0.865
Found a better model at epoch 1


Epoch #2: loss: 0.266, accuracy: 0.894
Found a better model at epoch 2


Epoch #3: loss: 0.252, accuracy: 0.902
Found a better model at epoch 3


Epoch #4: loss: 0.282, accuracy: 0.886


Epoch #5: loss: 0.268, accuracy: 0.895


...

Epoch #26: loss: 0.194, accuracy: 0.928


Epoch #27: loss: 0.199, accuracy: 0.925


Epoch #28: loss: 0.198, accuracy: 0.924


Epoch #29: loss: 0.181, accuracy: 0.932
Found a better model at epoch 29


Epoch #30: loss: 0.201, accuracy: 0.924


looks great with 93.2% accuracy. Now we have model on our disk space. It's not that useful sitting there, isn't it? So let's put it in a more of a warm *space*, a **hugging face** space. [Onto deployment!!!](https://huggingface.co/spaces/hazemamn/resnet18-Cat-vs-Dog-demo)